In [66]:
import sqlite3
con = sqlite3.connect('sql-murder-mystery.db')


In [2]:
import pandas as pd

pd.read_sql("SELECT * FROM sqlite_master", con)

,type,name,tbl_name,rootpage,sql
0,table,crime_scene_report,crime_scene_report,2,CREATE TABLE crime_scene_report (\n dat...
1,table,drivers_license,drivers_license,3,CREATE TABLE drivers_license (\n id int...
2,table,person,person,4,CREATE TABLE person (\n id integer PRIM...
3,table,facebook_event_checkin,facebook_event_checkin,5,CREATE TABLE facebook_event_checkin (\n ...
4,table,interview,interview,6,CREATE TABLE interview (\n person_id in...
5,table,get_fit_now_member,get_fit_now_member,7,CREATE TABLE get_fit_now_member (\n id ...
6,index,sqlite_autoindex_get_fit_now_member_1,get_fit_now_member,8,None
7,table,get_fit_now_check_in,get_fit_now_check_in,9,CREATE TABLE get_fit_now_check_in (\n m...
8,table,income,income,10,CREATE TABLE income (\n ssn integer PRI...
9,table,solution,solution,11,"CREATE TABLE solution (\n user integer,..."


In [3]:
query = """
    SELECT * FROM crime_scene_report
    WHERE (date == "20180115") and (city == "SQL City") and (type == "murder")
    
"""

pd.read_sql(query, con)['description'][0]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

In [4]:
# Suspect 1
## last house on "Northwestern Dr.

query = """
    SELECT P.name,
    I.transcript
    FROM person P
    LEFT JOIN interview I ON I.person_id = P.id
    WHERE P.address_street_name == "Northwestern Dr"
    ORDER BY P.address_number DESC
    LIMIT 1
"""
pd.read_sql(query, con)['transcript'][0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [5]:
# Suspect 2
## Annabel, lives somewhere on "Franklin Ave

query = """
    SELECT P.name,
    I.transcript
    FROM person P
    
    LEFT JOIN interview I ON I.person_id = P.id
    WHERE (address_street_name == "Franklin Ave") AND (name LIKE "%Annabel%")

"""
pd.read_sql(query, con)['transcript'][0]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

In [6]:
# CLUES
## "Get Fit Now Gym" bag
## Membership number on the bag started with "48Z"
## Gold members

## Car with a plate that included "H42W"
## Killer was working out last week on January the 9th.

query = """
    SELECT 
    M.id idGym,
    M.person_id,
    M.name,
    M.membership_status,

    P.license_id
    
    FROM get_fit_now_member M

    LEFT JOIN person P ON P.id = M.person_id

    WHERE (M.membership_status == "gold") AND (M.id LIKE "48Z%")
"""
pd.read_sql(query, con)

,idGym,person_id,name,membership_status,license_id
0,48Z7A,28819,Joe Germuska,gold,173289
1,48Z55,67318,Jeremy Bowers,gold,423327


In [16]:
## Car with a plate that included "H42W"
## Killer was working out last week on January the 9th.

query = """
    SELECT 
        *
    FROM person 
    WHERE (license_id == 173289) OR (license_id == 423327)
"""
pd.read_sql(query, con)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [74]:
query = """
    SELECT * 
    FROM interview 
    WHERE person_id == "28819" OR person_id == "67318"
"""
pd.read_sql(query,con)

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


In [32]:
# red hair and she drives a Tesla Model S.
query = """
    SELECT * 
    FROM drivers_license 
    WHERE (gender == "female") AND (hair_color == "red") AND (car_make == "Tesla") AND (car_model == "Model S")
"""
pd.read_sql(query,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [65]:
# she attended the SQL Symphony Concert 3 times in December 2017
query = """
    SELECT *,
    COUNT(person_id) times
    
    FROM facebook_event_checkin
    WHERE (event_name == "SQL Symphony Concert") AND (date LIKE "201712%")
    
    GROUP BY person_id    
    HAVING COUNT(person_id) = 3
    ORDER BY person_id

"""
pd.read_sql(query,con)

,person_id,event_id,event_name,date,times
0,24556,1143,SQL Symphony Concert,20171207,3
1,99716,1143,SQL Symphony Concert,20171206,3


In [73]:
# a woman with a lot of money
# Income
query = """
    SELECT * 
    FROM person P
    LEFT JOIN income I ON P.ssn = I.ssn
    WHERE id == "24556" OR id == "99716"
    """
pd.read_sql(query,con)

,id,name,license_id,address_number,address_street_name,ssn,ssn,annual_income
0,24556,Bryan Pardo,101191,703,Machine Ln,816663882,NaN,NaN
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,987756388.0,310000.0


In [75]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [76]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'